<a href="https://colab.research.google.com/gist/mmore500/a2e88e7c239935c362ec59c6b5a3f7b5/reconstruction-quality-experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Procedure:

For each experimental replicate per treatment,
- Navigate to <https://colab.research.google.com/gist/mmore500/a2e88e7c239935c362ec59c6b5a3f7b5> to open a fresh copy of the experiment notebook. **Open a fresh notebook copy for each treatment.**
- Click on filename on the top left of the Colab page(`a2e88e7c239935c362ec59c6b5a3f7b5`) and rename according to template
  - `evo=island{num_islands}-niche{num_niches}-ngen{num_generations}-popsize{population_size}-tournsize{tournament_size}+instrument={"steady"|"tilted"}-{"old"|"new"}-bits{annotation_size_bits}-diff{differentia_width}+replicate={replicate}+ext=.ipynb`.
  - For example, `evo=island1-niche1-ngen10000-popsize1024-tournsize2+instrument=steady-old-bits64-diff1+replicate=0+ext=.ipynb`.
- Configure variables in "Configure Experment" section.
- On the top menu, click `Runtime > Restart sesson and run all` if available, otherwise `Runtime > Run all`.
- Wait for final cell's execution to complete.
- Record configured variables and results from "Evaluate Reconstruction" section in [results spreadsheet](https://docs.google.com/spreadsheets/d/1ZhS4NDTDyBiwmwtWrZO5L06MGB3lhmp2-5ZzClhEwPU/edit?usp=sharing).
- On the top menu, click `File > Download > Download .ipynb`.
- Upload ipynb file to treatment directory at <https://osf.io/n4b2g/>, named same as notebook, except excluding `+replicate={replicate}+ext=.ipynb`.
  - Treatment directory should contain notebooks for each replicate of notebook.


## Set Up Environment

In [2]:
!python3 -m pip install \
    "alifedata_phyloinformatics_convert==0.15.1" \
    "biopython==1.83" \
    "dendropy==4.6.1" \
    "git+https://github.com/mmore500/hstrat-surface-concept.git@v0.1.0#egg=hsurf" \
    "hstrat==1.8.7" \
    "matplotlib==3.8.2" \
    "pandas==1.5.3" \
    "tqdist==1.0" \
    "tqdm==4.66.1" \
    "typing_extensions>=4.9.0" \
    "watermark==2.4.3"

In [3]:
from collections import Counter

import alifedata_phyloinformatics_convert as apc
from Bio import Phylo
import dendropy as dp
from hstrat import hstrat
from hstrat import _auxiliary_lib as hstrat_aux
from hsurf import hsurf
from matplotlib import pyplot as plt
import pandas as pd
import tqdist
from tqdm import tqdm

## Configure Experiment

Configure instrumentation. **Edit me**

In [4]:
# TODO Uncomment one...
# annotation_size_bits = 64
annotation_size_bits = 256
# annotation_size_bits = 1024
assert annotation_size_bits.bit_count() == 1, "must be power of 2 (1, 2, 4, 8, etc.)"

# TODO Uncomment one...
differentia_width_bits = 1
# differentia_width_bits = 8
assert differentia_width_bits.bit_count() == 1, "must be power of 2 (1, 2, 4, 8, etc.)"

# TODO Uncomment one...
stratum_retention_algo = hstrat.depth_proportional_resolution_tapered_algo  # old impl/steady behavior
# stratum_retention_algo = hstrat.recency_proportional_resolution_curbed_algo  # old impl/tilted behavior
# stratum_retention_algo = hsurf.stratum_retention_interop_steady_algo  # new impl/steady behavior
# stratum_retention_algo = hsurf.stratum_retention_interop_tilted_sticky_algo  # new impl/tilted behavior

Configure evolutionary scale. **Edit me**

In [5]:
# TODO Uncomment one...
# population_size = 1024  # default condition
population_size = 65536  # alternate condition
assert population_size.bit_count() == 1, "must be power of 2 (1, 2, 4, 8, etc.)"

# TODO Uncomment one...
num_generations = 10000  # default condition
# num_generations = 100000  # alternate condition


Configure evolutionary conditions.  **Edit me**

In [6]:
# TODO Uncomment one...
num_islands=1  # default condition
# num_islands=64  # alternate condition
assert num_islands.bit_count() == 1, "must be power of 2 (1, 2, 4, 8, etc.)"

# TODO Uncomment one...
num_niches=1  # default condition
# num_niches=8  # alternate condition
assert num_niches.bit_count() == 1, "must be power of 2 (1, 2, 4, 8, etc.)"

# TODO Uncomment one...
tournament_size=2  # default condition
# tournament_size=1  # alternate condition
# tournament_size=8  # alternate condition


Configure experimental replicate. **Edit me**

In [7]:
replicate = 0 # TODO set to a number, 0 through 19

Set up random number generator. (Do not edit.)

In [8]:
seed = hash(
  (
      replicate,
      population_size,
      num_generations,
      num_islands,
      num_niches,
      tournament_size,
  )
) % 2 ** 32

seed

3263141568

In [9]:
from hstrat._auxiliary_lib import seed_random

seed_random(seed)


Parametrize instrumentation. (Do not edit.)

In [10]:
annotation_capacity_strata = annotation_size_bits // differentia_width_bits
assert annotation_capacity_strata.bit_count() == 1, "must be power of 2 (1, 2, 4, 8, etc.)"
print(f"{annotation_capacity_strata=}")

parametrized_policy = stratum_retention_algo.Policy(
  parameterizer=hstrat.PropertyAtMostParameterizer(
    target_value=annotation_capacity_strata,
    policy_evaluator=hstrat.NumStrataRetainedUpperBoundEvaluator(
      at_num_strata_deposited=num_generations,
    ),
    param_lower_bound=2,
    param_upper_bound=1024,
  ),
)

print(f"{parametrized_policy=}")
print(f"num strata retained upper bound {parametrized_policy.CalcNumStrataRetainedUpperBound(num_generations)}")


annotation_capacity_strata=256
parametrized_policy=depth_proportional_resolution_tapered_algo.Policy(policy_spec=depth_proportional_resolution_tapered_algo.PolicySpec(depth_proportional_resolution=127))
num strata retained upper bound 255


## Setup

Helper functions.

In [11]:
def tree_quartet_distance(
    x,
    y,
    progress_wrap: typing.Callable = lambda x: x,
  ) -> float:
    """Calculate dissimilarity between two trees. Used to measure how accurate
    tree reconstructions are."""
    tree_a = apc.RosettaTree(x).as_dendropy
    tree_b = apc.RosettaTree(y).as_dendropy

    # must suppress root unifurcations or tqdist barfs
    # see https://github.com/uym2/tripVote/issues/15
    tree_a.unassign_taxa(exclude_leaves=True)
    tree_a.suppress_unifurcations()
    tree_b.unassign_taxa(exclude_leaves=True)
    tree_b.suppress_unifurcations()

    tree_a_taxon_labels = [
        leaf.taxon.label for leaf in progress_wrap(tree_a.leaf_node_iter())
    ]
    tree_b_taxon_labels = [
        leaf.taxon.label for leaf in progress_wrap(tree_b.leaf_node_iter())
    ]
    all(
        progress_wrap(
          zip(tree_a.leaf_node_iter(), tree_b.leaf_node_iter(), strict=True),
        ),
    )
    assert sorted(tree_a_taxon_labels) == sorted(tree_b_taxon_labels), (
        tree_a_taxon_labels,
        tree_b_taxon_labels,
    )
    for taxon_label in progress_wrap(tree_a_taxon_labels):
        assert taxon_label
        assert taxon_label.strip()

    newick_a = tree_a.as_string(schema="newick").strip()
    newick_b = tree_b.as_string(schema="newick").strip()

    return tqdist.quartet_distance(newick_a, newick_b)


In [12]:
def count_inner_nodes(df: pd.DataFrame) -> int:
  """Count how many non-leaf nodes are contained in phylogeny."""
  df = hstrat_aux.alifestd_collapse_unifurcations(df)
  num_leaves = len(hstrat_aux.alifestd_find_leaf_ids(df))
  res = len(df) - num_leaves
  assert 0 <= res < num_leaves
  return res

In [13]:
def calc_polytomic_index(df: pd.DataFrame) -> int:
  """Count how many fewer inner nodes are contained in phylogeny than expected
  if strictly bifurcationg."""
  df = hstrat_aux.alifestd_collapse_unifurcations(df)
  num_leaves = len(hstrat_aux.alifestd_find_leaf_ids(df))
  expected_rows_if_bifurcating = 2 * num_leaves - 1
  res = expected_rows_if_bifurcating - len(df)
  assert 0 <= res < num_leaves
  return res

In [14]:
def count_polytomies(df: pd.DataFrame) -> int:
  """Count how many inner nodes have more than two descendant nodes."""
  df = hstrat_aux.alifestd_collapse_unifurcations(df)
  ancestor_counts = Counter(df["ancestor_id"])
  return sum(
      v > 2 for v in ancestor_counts.values()
  )


## Generate Phylogeny

Use simple evolutionary simulation to generate a phylogenetic history to test reconstruction process on.

In [15]:
true_phylogeny_df = hstrat.evolve_fitness_trait_population(
    num_islands=num_islands,
    num_niches=num_niches,
    num_generations=num_generations,
    population_size=population_size,
    tournament_size=tournament_size,
    progress_wrap=tqdm,
)
true_phylogeny_df["taxon_label"] = true_phylogeny_df["loc"].astype(str)
true_phylogeny_df

100%|██████████| 212065/212065 [00:02<00:00, 77444.79it/s]


,id,ancestor_list,loc,trait,origin_time,island,niche,taxon_label
0,0,[None],0,NaN,0,0,0,0
1,1,[0],12734,0.000000,1,0,0,12734
2,2,[1],54082,2.316256,2,0,0,54082
3,3,[2],41704,4.521688,3,0,0,41704
4,4,[3],2263,5.110847,4,0,0,2263
...,...,...,...,...,...,...,...,...
212060,212060,[119406],65531,11401.050781,10001,0,0,65531
212061,212061,[143933],65532,11397.212891,10001,0,0,65532
212062,212062,[110569],65533,11399.445312,10001,0,0,65533
212063,212063,[120372],65534,11395.846680,10001,0,0,65534


## Generate Reconstruction

Generate genome annotations as if tracking phylogeny in distributed environment.
Then run reconstruction proess to estimate true phylogeny from generated annotations.

In [18]:
extant_annotations = hstrat.descend_template_phylogeny_alifestd(
    hstrat_aux.alifestd_collapse_unifurcations(true_phylogeny_df),
    seed_column=hstrat.HereditaryStratigraphicColumn(parametrized_policy),
    progress_wrap=tqdm,
)

len(extant_annotations)


100%|██████████| 110198/110198 [00:05<00:00, 18824.12it/s]

100%|██████████| 65536/65536 [00:25<00:00, 2554.40it/s]


65536

In [19]:
reconstructed_phylogeny_df = hstrat.build_tree(
  extant_annotations,
  progress_wrap=tqdm,
  version_pin=hstrat.__version__,
)
reconstructed_phylogeny_df


100%|██████████| 65536/65536 [01:26<00:00, 755.45it/s]

0it [00:00, ?it/s]
9578it [00:00, 95767.31it/s]
19155it [00:00, 91030.83it/s]
28274it [00:00, 88287.64it/s]
37115it [00:00, 82713.80it/s]
45606it [00:00, 83450.12it/s]
54939it [00:00, 86658.86it/s]
65536it [00:00, 85817.90it/s]

0it [00:00, ?it/s]
3273it [00:00, 32721.37it/s]
6618it [00:00, 33116.42it/s]
9930it [00:00, 29107.54it/s]
12892it [00:00, 27398.36it/s]
16015it [00:00, 28649.19it/s]
18915it [00:00, 28415.35it/s]
21779it [00:00, 28158.75it/s]
24837it [00:00, 28866.49it/s]
27737it [00:00, 28785.18it/s]
30625it [00:01, 28548.74it/s]
33565it [00:01, 28799.44it/s]
38568it [00:01, 35148.80it/s]
43529it [00:01, 39477.88it/s]
48173it [00:01, 41561.25it/s]
53736it [00:01, 45776.33it/s]
59272it [00:01, 48642.95it/s]
64145it [00:01, 48366.68it/s]
69408it [00:01, 49637.12it/s]
75304it [00:01, 52395.34it/s]
81571it [00:02, 55467.71it/s]
87523it [00:02, 56676.75it/s]
93194it [00:02, 56525.52it/s]
99075it [00:02, 57207.16it/s]
104798it

,id,ancestor_list,origin_time,taxon_label,ancestor_id
0,0,[none],0.0,Root,0
248,248,[0],9839.5,Inner+r=9824+d=NKuxzRacIGJ+uid=Bv6lHKT64s9c4z3...,0
249,249,[248],9871.5,Inner+r=9856+d=J7_2xWWp6K6+uid=DCNEcvW1h5aq1hG...,248
252,252,[249],9903.5,Inner+r=9888+d=cDKQuFrGcD+uid=COkD_ZNDO2DGHkBq...,249
261,261,[248],9903.5,Inner+r=9888+d=EVB0NvaLRJR+uid=D80m2t_NLPHrZOF...,248
...,...,...,...,...,...
132206,132206,[1137],10001.0,27237,1137
132207,132207,[1137],10001.0,34504,1137
132208,132208,[1137],10001.0,44508,1137
132209,132209,[1137],10001.0,50809,1137


## Evaluate Reconstruction

Reconstruction quality data --- collect into spreadsheet.

In [24]:
reconstruction_error = tree_quartet_distance(
  true_phylogeny_df,
  reconstructed_phylogeny_df,
  progress_wrap=tqdm,
)
f"{reconstruction_error=}"

TypeError: tree_quartet_distance() got an unexpected keyword argument 'progress_wrap'

In [21]:
num_true_inner_nodes = count_inner_nodes(true_phylogeny_df)
num_reconstructed_inner_nodes = count_inner_nodes(reconstructed_phylogeny_df)
f"{num_true_inner_nodes=} {num_reconstructed_inner_nodes=}"

'num_true_inner_nodes=44662 num_reconstructed_inner_nodes=792'

In [22]:
num_true_polytomies = count_polytomies(true_phylogeny_df)
num_reconstructed_polytomies = count_polytomies(reconstructed_phylogeny_df)
f"{num_true_polytomies=} {num_reconstructed_polytomies=}"

'num_true_polytomies=14818 num_reconstructed_polytomies=728'

In [23]:
true_polytomic_index = calc_polytomic_index(true_phylogeny_df)
reconstructed_polytomic_index = calc_polytomic_index(reconstructed_phylogeny_df)
f"{true_polytomic_index=} {reconstructed_polytomic_index=}"

'true_polytomic_index=20873 reconstructed_polytomic_index=64743'

## Visualize Phylogeny & Reconstruction

For validating results.

Topology only (no time).

In [ ]:
true_phylogeny_tree = apc.alife_dataframe_to_biopython_tree(
  hstrat_aux.alifestd_collapse_unifurcations(true_phylogeny_df),
  setup_branch_lengths=False,
)
reconstructed_phylogeny_tree = apc.alife_dataframe_to_biopython_tree(
  hstrat_aux.alifestd_collapse_unifurcations(reconstructed_phylogeny_df),
  setup_branch_lengths=False,
)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

ax1.set_title("True Tree")
Phylo.draw(true_phylogeny_tree, do_show=False, axes=ax1)

ax2.set_title("Reconstructed Tree")
Phylo.draw(reconstructed_phylogeny_tree, do_show=False, axes=ax2)

plt.tight_layout()
plt.show()

Scaled by time.

In [ ]:
true_phylogeny_tree = apc.alife_dataframe_to_biopython_tree(
  hstrat_aux.alifestd_collapse_unifurcations(true_phylogeny_df),
  setup_branch_lengths=True,
)
reconstructed_phylogeny_tree = apc.alife_dataframe_to_biopython_tree(
  hstrat_aux.alifestd_collapse_unifurcations(reconstructed_phylogeny_df),
  setup_branch_lengths=True,
)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

ax1.set_title("True Tree")
ax1.set_xscale("log")
Phylo.draw(true_phylogeny_tree, do_show=False, axes=ax1)

ax2.set_title("Reconstructed Tree")
ax2.set_xscale("log")
Phylo.draw(reconstructed_phylogeny_tree, do_show=False, axes=ax2)

plt.tight_layout()
plt.show()

## Reproducibility Information

For future reference if reproducing experiments.

In [ ]:
print(
  f"""# instrumentation
{annotation_size_bits=}
{differentia_width_bits=}
{stratum_retention_algo.PolicySpec.GetAlgoTitle()=}

# evolutionary scale
{population_size=}
{num_generations=}

# evolutionary conditions
{num_islands=}
{num_niches=}
{tournament_size=}
"""
)

In [ ]:
import datetime
datetime.datetime.now().isoformat()

In [ ]:
%load_ext watermark
%watermark

In [ ]:
!pip freeze

In [ ]:
locals()